In [ ]:
%cd /content
!git clone https://github.com/arenatemp/sd-inference-server.git
%cd /content/sd-inference-server

model_folder = "./models"
try:
    from google.colab import drive
    import os
    drive.mount('/content/drive')
    model_folder = "/content/drive/My Drive/qDiffusion/models"
    !mkdir '/content/drive/My Drive/qDiffusion' -p
    if not os.path.exists('/content/drive/My Drive/qDiffusion/models'):
        !cp -r 'models/' '/content/drive/My Drive/qDiffusion/'
except Exception as e:
    pass

!gdown "10JxD-Bnd8g9XMFoKj8EK6nkPyRg2g0r0&confirm=t" -O cached_venv.zip
import zipfile
with zipfile.ZipFile("cached_venv.zip", 'r') as zip_ref:
    zip_ref.extractall("venv")
import sys
sys.path.insert(0, "/content/sd-inference-server/venv/lib/python3.9/site-packages")
sys.path.insert(0, "/content/sd-inference-server/")

import random
import torch
import attention
import storage
import wrapper
import string
from server import Server

attention.use_optimized_attention()

model_storage = storage.ModelStorage(model_folder, torch.float16, torch.float32)
params = wrapper.GenerationParameters(model_storage, torch.device("cuda"))

password = ''.join(random.SystemRandom().choice(string.ascii_letters + string.digits) for _ in range(8))

server = Server(params, "127.0.0.1", "28888", password)
server.start()

from pycloudflared import try_cloudflare
tunnel_url = try_cloudflare(port=28888, verbose=False)
print("ENDPOINT: ", tunnel_url.tunnel.replace("https", "wss"))
print("PASSWORD: ", password)

In [ ]:
server.stop()
drive.mount('/content/drive', force_remount=True)
server = Server(params, "127.0.0.1", "28888", password)
server.start()